# AC 221: Critical Thinking in Data Science

## Problem Set 1, Part 2
**February 27, 2020**<br/>
**Harvard University**<br/>
**Spring 2020**<br/>

<hr style="height:2pt">

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from azureml.opendatasets import UsPopulationZip

ModuleNotFoundError: No module named 'azureml'

## Problem 1

### Data Sources
#### US Census FactFinder
The distribution of age was calculating using US 2010 Decennial Census.

Annual Estimates of the Resident Population by Single Year of Age and Sex for the United States: April 1, 2010 to July 1, 2018
Source: U.S. Census Bureau, Population Division
Release Date: June 2019 
Link: https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk

#### Microsoft Azure - Open Datasets
The distributions for zipcode, sex, and gender were calculated using Microsoft Azure's *US Population by Zipcode* Open Dataset, which in turn was sourced from the 2010 US Decennial Census. Because the census collects data only on sex and not gender, we used sex as a proxy for gender.

Link: https://azure.microsoft.com/en-us/services/open-datasets/catalog/us-decennial-census-zip/

### Age

In [ ]:
age_df = pd.read_csv("data/census_age_0-100.csv")
age_2010 = age_df[['Age', '2010']][1:]

In [ ]:
xticks = list(range(0, 100, 5))
xtick_labels = [str(x) for x in xticks]
xticks.append(100)
xtick_labels.append('100+')

plt.figure(figsize=(10,10))
plt.title("Figure 1.1: Histogram of Age in United States, 2010")
plt.bar(age_2010['Age'], age_2010['2010'], width = 1.0)
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.xticks(xticks, xtick_labels)
plt.show()

As seen in **Figure 1.1**, age is not uniformly distributed in the United States. Instead, it has a bimodal, right skewed distribution. 

### Zip Code

In [ ]:
population_df = UsPopulationZip().to_pandas_dataframe()

In [ ]:
# Remove rows with None's, which are aggregations of the more specific values.
pop = population_df.query('~(sex.isnull()) & (~race.isnull()) & ~(minAge.isna() | maxAge.isna())', engine='python')

In [ ]:
# Graph
ax = pop.groupby('zipCode')['population'].sum().plot.hist(bins=50, logy=True)
ax.set_title('Figure 1.2: Histogram of Log Pop. Counts within Zipcodes, 2010')
ax.set_xlabel('Pop. Size')

**Figure 1.2** shows that the population size within zipcodes is not uniformly distributed.

### Gender (By Proxy of Sex)

In [ ]:
sex_ratios = pop.groupby(['zipCode']).apply((lambda g: g.query('sex == "Female"')['population'].sum()
                                       / g.query('sex == "Male"')['population'].sum()))
np.random.seed(1)
ax = sex_ratios.replace([np.inf, -np.inf], np.nan).dropna().sample(1000).plot.hist(bins=30)
ax.set_xlabel("Female to Male Population Ratio")
ax.set_title("Figure 1.3: Histogram of Female to Male Population Ratio by Zipcode, 2010")

Interpretation.

### Race

In [ ]:
cambridge = pop.query('zipCode == "02138"')
ax = pd.DataFrame({
    'US Total': pop.groupby('race')['population'].sum() / 
                pop['population'].sum(),
    'Cambridge (02138)': cambridge.groupby('race')['population'].sum() / 
                         cambridge['population'].sum(),
}).plot.bar(logy=True)
ax.set_title('Figure 1.4: US vs. Cambridge Race Proportions')
ax.set_ylabel('Proportion of Total Population (log scale)')

Interpretation.

## General edX Data Processing

In [2]:
quasi = pd.read_csv('data/final_quasi-identifiers.csv')
quasi_list = list(quasi['variable'].values)

# Drop user_id from quasi_list
quasi_list.remove('user_id')

In [3]:
df = pd.read_csv('APCOMP221Pset1.csv').drop(['username', 'ip', 
                                             'un_major_region', 'un_economic_group', 
                                             'un_developing_nation', 'un_special_region',
                                             'latitude', 'longitude', 'profile_country'], axis=1)

/home/pau/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (49,50,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Read small dataset for writing functions
df = pd.read_csv('APCOMP221Pset1.csv', nrows = 10000).drop(['username', 'ip', 
                                             'un_major_region', 'un_economic_group', 
                                             'un_developing_nation', 'un_special_region',
                                             'latitude', 'longitude', 'profile_country'], axis=1)

### Replacing NaNs

In [4]:
na_replacements = {
    'course_id': 'NA',
    'cc_by_ip': 'NA',
    'countryLabel': 'NA',
    'continent': 'NA',
    'city': 'NA',
    'region': 'NA',
    'subdivision': 'NA',
    'postalCode': 'NA',
    'LoE': 'NA',
    'YoB': -1,
    'gender': 'NA',
    'nforum_posts': 0,
    'nforum_votes': 0,
    'nforum_endorsed': 0,
    'nforum_threads': 0,
    'nforum_comments': 0,
    'nforum_pinned': 0,
    'nforum_events': 0,
    'mode': 'NA',
    'email_domain': 'NA'
}

df = df.fillna(na_replacements)

df.head()

,course_id,user_id,registered,viewed,explored,certified,completed,cc_by_ip,countryLabel,continent,...,roles_isInstructor,roles_isStaff,roles_isCCX,roles_isFinance,roles_isLibrary,roles_isSales,forumRoles_isAdmin,forumRoles_isCommunityTA,forumRoles_isModerator,forumRoles_isStudent
0,HarvardX/PH525.1x/1T2018,1488411,True,False,NaN,False,False,GB,United Kingdom,Europe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,HarvardX/PH525.1x/1T2018,7013084,True,True,True,False,False,US,United States,Americas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,HarvardX/PH525.1x/1T2018,4083257,True,True,False,False,False,NA,NA,NA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,HarvardX/PH525.1x/1T2018,4605571,True,True,False,False,False,US,United States,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,HarvardX/PH525.1x/1T2018,1499820,True,True,True,False,False,HU,Hungary,Europe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


## Problem 3

### Useful Functions

In [5]:
def suppress_below_k(k):
    groups = df.groupby(quasi_list).size()
    groups_below_threshold = groups[groups < k]
    return groups[groups < k]

def suppressed_n(groups):
    return groups_below_threshold.sum()

def percent_remaining(groups):
    return (1 - groups_below_threshold.sum() / df.shape[0]) * 100

def suppress_to_k_anonymize(df, quasi_ids, k):
    return df.groupby(quasi_ids).filter(lambda g: g.shape[0] >= k)

def status_count(df, status):
    return df.query(status + ' == True').shape[0]

def status_proportion(df, status):
    return status_count(df, status) / df.shape[0] * 100

def print_status_stats(df):
    print(f'Total students: {df.shape[0]}\n'
          f'Students explored: {status_count(df, "explored")}'
          f' - {status_proportion(df, "explored"):.2f}%\n'
          f'Students completed: {status_count(df, "completed")}'
          f' - {status_proportion(df, "completed"):.2f}%')

### Preliminary Check of User ID

In [23]:
### Checking if any user takes the same course more than once
sizes = df.groupby('user_id')['course_id'].size()
unique_sizes = df.groupby('user_id')['course_id'].nunique()
compare = (sizes == unique_sizes)
same_class = compare[compare == True]
same_class.value_counts()

True    936434
Name: course_id, dtype: int64

It does not appear that any user is taking the same class more than once, which implies that all $n$ records for the same quasi-identifier are from different users. 

### Raw Dataset Stats

In [8]:
print_status_stats(df)

Total students: 1141735
Students explored: 122258 - 10.71%
Students completed: 28142 - 2.46%


### 3-Anonymous Results

In [18]:
groups_below_threshold = suppress_below_k(3)

print("k = 3")
print("N suppresed: {}".format(suppressed_n(groups_below_threshold)))
print("Percent remaining: {}".format(percent_remaining(groups_below_threshold)))

k = 3
N suppresed: 988191
Percent remaining: 13.448304554033996


In [ ]:
k3_df = suppress_to_k_anonymize(df, quasi_list, 3)
print_status_stats(k3_df)

When making the quasi-identifier dataset k-anonymous using suppression for $k=3$, 988191 records are suppressed. This leaves **13.45%** of the 1141735 records in the original dataset. 

The completion rate is X. The exploration rate is Y. 

### 4-Anonymous Results

In [20]:
# k = 4
groups_below_threshold = suppress_below_k(4)

print("k = 4")
print("N suppresed: {}".format(suppressed_n(groups_below_threshold)))
print("Percent remaining: {}".format(percent_remaining(groups_below_threshold)))

k = 4
N suppresed: 1010127
Percent remaining: 11.527018090887985


In [ ]:
k4_df = suppress_to_k_anonymize(df, quasi_list, 4)
print_status_stats(k3_df)

When making the quasi-identifier dataset k-anonymous using suppression for $k=4$, 1010127 records are suppressed. This leaves **11.53%** of the 1141735 records in the original dataset. 

The completion rate is X. The exploration rate is Y.

In [ ]:
# k = 5
groups_below_threshold = suppress_below_k(5)

print("k = 5")
print("N suppresed: {}".format(suppressed_n(groups_below_threshold)))
print("Percent remaining: {}".format(percent_remaining(groups_below_threshold)))

In [ ]:
k4_df = suppress_to_k_anonymize(df, quasi_list, 5)
print_status_stats(k3_df)

When making the quasi-identifier dataset k-anonymous using suppression for $k=4$, 1023587 records are suppressed. This leaves **10.35%** of the 1141735 records in the original dataset. 

The completion rate is X. The exploration rate is Y.

## Question 4

### Useful Functions

In [6]:
def n_synthetic_required_for(k):
    groups = df.groupby(quasi_list).size()
    groups_below_threshold = groups[groups < k]
    return (k - groups_below_threshold).sum()

def suppressed_records(df, quasi_ids, k):
    return df.groupby(quasi_ids).filter(lambda g: g.shape[0] < k)

def free_ids(df, n):
    # Create list of unused IDs for synthetic records
    user_ids = df['user_id'].unique()
    user_ids.sort()

    range_ids = np.arange(1,10000001, 1)
    mask = np.isin(range_ids, user_ids, invert = True)
    free_ids = range_ids[mask]
    return random.choices(free_ids, k = n)

def marginal_variable_values(df, variable, n):
    # Create list of random values for desired variable using distribution from full dataset
    counts = df[variable].value_counts()
    return random.choices(counts.index.values, weights=counts.values, k=n)

def variables_for_synthetic(variables):
    # Create list of all variables in synthetic dataset: only care about quasi-identifiers, user_id, completed, and explored
    suppressed_vars = quasi_list.copy()
    for var in variables:
        suppressed_vars.append(var)
    return suppressed_vars

def suppressed_multi_index(df, quasi_identifiers, k):
    # Get multi-index
    groups = df.groupby(quasi_identifiers).size()
    return groups[groups < k]

def split_df(df, variables_for_synthetic, quasi_identifiers, k):
    # Split dataframe into suppressed and non-suppressed records
    suppressed_df = suppressed_records(df[variables_for_synthetic], quasi_list, k)
    nonsuppressed_df = suppress_to_k_anonymize(df[variables_for_synthetic], quasi_list, k)
    return suppressed_df, nonsuppressed_df

def synthetic_df(df, non_quasi_variables, quasi_identifiers, k):
    # Setup
    synth_n = n_synthetic_required_for(k)
    all_synth_vars = variables_for_synthetic(non_quasi_variables)
    sup_df, nonsup_df = split_df(df, all_synth_vars, quasi_identifiers, k)
    
    # Random lists
    random_user_ids = free_ids(df, synth_n)
    random_completed = marginal_variable_values(df, 'completed', synth_n)
    random_explored = marginal_variable_values(df, 'explored', synth_n)
    
    # Indices setup for synthetic loop
    groups_below_threshold = suppressed_multi_index(df, quasi_identifiers, k)
    sup_df.set_index(quasi_list, inplace = True)
    index_tuples = tuple(zip(groups_below_threshold, groups_below_threshold.index))

    # Loop for creating synthetic records
    for t in index_tuples:
        t_size = t[0]

        # Add synthetic records
        for s in range(0, k-t_size):
            # Create synthetic variables
            user_id = random_user_ids.pop(0)
            completed = random_completed.pop(0)
            explored = random_explored.pop(0)

            # Create row
            row = list(t)
            row.append(user_id)
            row.append(completed)
            row.append(explored)
            row_s = pd.Series(row)
            sup_df = sup_df.append(row_s, ignore_index = True)
    
    # Concat suppressed (with added synthetic rows) and non-suppressed dataframes
    synth_df = nonsup_df.append(sup_df)
    
    # Randomize row order 
    return synth_df.sample(frac=1).reset_index(drop=True)

### Creating Synthetic Rows for Database
For every record whose quasi-identifiers appeared in  $n$ < $k$ records, we generated $k - n$ synthetic records.

As we are only concerned with comparing `completed` and `explored` rates, we only included these variables in addition to the quasi-identifiers and `user_id`. 

A fake `user_id` was created by randomly selecting a n unused number in the range [1, 1000000]. This was the range of numbers for real ids. This means all synthetic records represent users who did not take more than one class. Ideally, we would sample user_ids in a way that distribution of number of classes taken by a user. 

The values for `explored` and `completed` were drawn using the marginal distribution. Thus, they were each independently drawn according to the distribution of the corresponding column in the original dataset. Because our two variables are binary categories, the distributions were Bernoulli with proportions of success easy to calculate. The marginal distribution was chosen because it gives a good mean vs correlation bias trade-off while not being as computationally expensive as using the marginal mean distribution.

### 3-Anonymous Results

In [7]:
synth_k3_df = synthetic_df(df, ['user_id', 'completed', 'explored'], quasi_list, 3)
print_status_stats(synth_k3_df)

(9248, 8)
(9249, 8)
(9250, 8)
(9251, 8)
(9252, 8)
(9253, 8)
(9254, 8)
(9255, 8)
(9256, 8)
(9257, 8)
(9258, 8)
(9259, 8)
(9260, 8)
(9261, 8)
(9262, 8)
(9263, 8)
(9264, 8)
(9265, 8)
(9266, 8)
(9267, 8)
(9268, 8)
(9269, 8)
(9270, 8)
(9271, 8)
(9272, 8)
(9273, 8)
(9274, 8)
(9275, 8)
(9276, 8)
(9277, 8)
(9278, 8)
(9279, 8)
(9280, 8)
(9281, 8)
(9282, 8)
(9283, 8)
(9284, 8)
(9285, 8)
(9286, 8)
(9287, 8)
(9288, 8)
(9289, 8)
(9290, 8)
(9291, 8)
(9292, 8)
(9293, 8)
(9294, 8)
(9295, 8)
(9296, 8)
(9297, 8)
(9298, 8)
(9299, 8)
(9300, 8)
(9301, 8)
(9302, 8)
(9303, 8)
(9304, 8)
(9305, 8)
(9306, 8)
(9307, 8)
(9308, 8)
(9309, 8)
(9310, 8)
(9311, 8)
(9312, 8)
(9313, 8)
(9314, 8)
(9315, 8)
(9316, 8)
(9317, 8)
(9318, 8)
(9319, 8)
(9320, 8)
(9321, 8)
(9322, 8)
(9323, 8)
(9324, 8)
(9325, 8)
(9326, 8)
(9327, 8)
(9328, 8)
(9329, 8)
(9330, 8)
(9331, 8)
(9332, 8)
(9333, 8)
(9334, 8)
(9335, 8)
(9336, 8)
(9337, 8)
(9338, 8)
(9339, 8)
(9340, 8)
(9341, 8)
(9342, 8)
(9343, 8)
(9344, 8)
(9345, 8)
(9346, 8)
(9347, 8)


(10068, 8)
(10069, 8)
(10070, 8)
(10071, 8)
(10072, 8)
(10073, 8)
(10074, 8)
(10075, 8)
(10076, 8)
(10077, 8)
(10078, 8)
(10079, 8)
(10080, 8)
(10081, 8)
(10082, 8)
(10083, 8)
(10084, 8)
(10085, 8)
(10086, 8)
(10087, 8)
(10088, 8)
(10089, 8)
(10090, 8)
(10091, 8)
(10092, 8)
(10093, 8)
(10094, 8)
(10095, 8)
(10096, 8)
(10097, 8)
(10098, 8)
(10099, 8)
(10100, 8)
(10101, 8)
(10102, 8)
(10103, 8)
(10104, 8)
(10105, 8)
(10106, 8)
(10107, 8)
(10108, 8)
(10109, 8)
(10110, 8)
(10111, 8)
(10112, 8)
(10113, 8)
(10114, 8)
(10115, 8)
(10116, 8)
(10117, 8)
(10118, 8)
(10119, 8)
(10120, 8)
(10121, 8)
(10122, 8)
(10123, 8)
(10124, 8)
(10125, 8)
(10126, 8)
(10127, 8)
(10128, 8)
(10129, 8)
(10130, 8)
(10131, 8)
(10132, 8)
(10133, 8)
(10134, 8)
(10135, 8)
(10136, 8)
(10137, 8)
(10138, 8)
(10139, 8)
(10140, 8)
(10141, 8)
(10142, 8)
(10143, 8)
(10144, 8)
(10145, 8)
(10146, 8)
(10147, 8)
(10148, 8)
(10149, 8)
(10150, 8)
(10151, 8)
(10152, 8)
(10153, 8)
(10154, 8)
(10155, 8)
(10156, 8)
(10157, 8)
(10158, 8)

(10815, 8)
(10816, 8)
(10817, 8)
(10818, 8)
(10819, 8)
(10820, 8)
(10821, 8)
(10822, 8)
(10823, 8)
(10824, 8)
(10825, 8)
(10826, 8)
(10827, 8)
(10828, 8)
(10829, 8)
(10830, 8)
(10831, 8)
(10832, 8)
(10833, 8)
(10834, 8)
(10835, 8)
(10836, 8)
(10837, 8)
(10838, 8)
(10839, 8)
(10840, 8)
(10841, 8)
(10842, 8)
(10843, 8)
(10844, 8)
(10845, 8)
(10846, 8)
(10847, 8)
(10848, 8)
(10849, 8)
(10850, 8)
(10851, 8)
(10852, 8)
(10853, 8)
(10854, 8)
(10855, 8)
(10856, 8)
(10857, 8)
(10858, 8)
(10859, 8)
(10860, 8)
(10861, 8)
(10862, 8)
(10863, 8)
(10864, 8)
(10865, 8)
(10866, 8)
(10867, 8)
(10868, 8)
(10869, 8)
(10870, 8)
(10871, 8)
(10872, 8)
(10873, 8)
(10874, 8)
(10875, 8)
(10876, 8)
(10877, 8)
(10878, 8)
(10879, 8)
(10880, 8)
(10881, 8)
(10882, 8)
(10883, 8)
(10884, 8)
(10885, 8)
(10886, 8)
(10887, 8)
(10888, 8)
(10889, 8)
(10890, 8)
(10891, 8)
(10892, 8)
(10893, 8)
(10894, 8)
(10895, 8)
(10896, 8)
(10897, 8)
(10898, 8)
(10899, 8)
(10900, 8)
(10901, 8)
(10902, 8)
(10903, 8)
(10904, 8)
(10905, 8)

(11577, 8)
(11578, 8)
(11579, 8)
(11580, 8)
(11581, 8)
(11582, 8)
(11583, 8)
(11584, 8)
(11585, 8)
(11586, 8)
(11587, 8)
(11588, 8)
(11589, 8)
(11590, 8)
(11591, 8)
(11592, 8)
(11593, 8)
(11594, 8)
(11595, 8)
(11596, 8)
(11597, 8)
(11598, 8)
(11599, 8)
(11600, 8)
(11601, 8)
(11602, 8)
(11603, 8)
(11604, 8)
(11605, 8)
(11606, 8)
(11607, 8)
(11608, 8)
(11609, 8)
(11610, 8)
(11611, 8)
(11612, 8)
(11613, 8)
(11614, 8)
(11615, 8)
(11616, 8)
(11617, 8)
(11618, 8)
(11619, 8)
(11620, 8)
(11621, 8)
(11622, 8)
(11623, 8)
(11624, 8)
(11625, 8)
(11626, 8)
(11627, 8)
(11628, 8)
(11629, 8)
(11630, 8)
(11631, 8)
(11632, 8)
(11633, 8)
(11634, 8)
(11635, 8)
(11636, 8)
(11637, 8)
(11638, 8)
(11639, 8)
(11640, 8)
(11641, 8)
(11642, 8)
(11643, 8)
(11644, 8)
(11645, 8)
(11646, 8)
(11647, 8)
(11648, 8)
(11649, 8)
(11650, 8)
(11651, 8)
(11652, 8)
(11653, 8)
(11654, 8)
(11655, 8)
(11656, 8)
(11657, 8)
(11658, 8)
(11659, 8)
(11660, 8)
(11661, 8)
(11662, 8)
(11663, 8)
(11664, 8)
(11665, 8)
(11666, 8)
(11667, 8)

(12344, 8)
(12345, 8)
(12346, 8)
(12347, 8)
(12348, 8)
(12349, 8)
(12350, 8)
(12351, 8)
(12352, 8)
(12353, 8)
(12354, 8)
(12355, 8)
(12356, 8)
(12357, 8)
(12358, 8)
(12359, 8)
(12360, 8)
(12361, 8)
(12362, 8)
(12363, 8)
(12364, 8)
(12365, 8)
(12366, 8)
(12367, 8)
(12368, 8)
(12369, 8)
(12370, 8)
(12371, 8)
(12372, 8)
(12373, 8)
(12374, 8)
(12375, 8)
(12376, 8)
(12377, 8)
(12378, 8)
(12379, 8)
(12380, 8)
(12381, 8)
(12382, 8)
(12383, 8)
(12384, 8)
(12385, 8)
(12386, 8)
(12387, 8)
(12388, 8)
(12389, 8)
(12390, 8)
(12391, 8)
(12392, 8)
(12393, 8)
(12394, 8)
(12395, 8)
(12396, 8)
(12397, 8)
(12398, 8)
(12399, 8)
(12400, 8)
(12401, 8)
(12402, 8)
(12403, 8)
(12404, 8)
(12405, 8)
(12406, 8)
(12407, 8)
(12408, 8)
(12409, 8)
(12410, 8)
(12411, 8)
(12412, 8)
(12413, 8)
(12414, 8)
(12415, 8)
(12416, 8)
(12417, 8)
(12418, 8)
(12419, 8)
(12420, 8)
(12421, 8)
(12422, 8)
(12423, 8)
(12424, 8)
(12425, 8)
(12426, 8)
(12427, 8)
(12428, 8)
(12429, 8)
(12430, 8)
(12431, 8)
(12432, 8)
(12433, 8)
(12434, 8)

...

(23663, 8)
(23664, 8)
(23665, 8)
(23666, 8)
(23667, 8)
(23668, 8)
(23669, 8)
(23670, 8)
(23671, 8)
(23672, 8)
(23673, 8)
(23674, 8)
(23675, 8)
(23676, 8)
(23677, 8)
(23678, 8)
(23679, 8)
(23680, 8)
(23681, 8)
(23682, 8)
(23683, 8)
(23684, 8)
(23685, 8)
(23686, 8)
(23687, 8)
(23688, 8)
(23689, 8)
(23690, 8)
(23691, 8)
(23692, 8)
(23693, 8)
(23694, 8)
(23695, 8)
(23696, 8)
(23697, 8)
(23698, 8)
(23699, 8)
(23700, 8)
(23701, 8)
(23702, 8)
(23703, 8)
(23704, 8)
(23705, 8)
(23706, 8)
(23707, 8)
(23708, 8)
(23709, 8)
(23710, 8)
(23711, 8)
(23712, 8)
(23713, 8)
(23714, 8)
(23715, 8)
(23716, 8)
(23717, 8)
(23718, 8)
(23719, 8)
(23720, 8)
(23721, 8)
(23722, 8)
(23723, 8)
(23724, 8)
(23725, 8)
(23726, 8)
(23727, 8)
(23728, 8)
(23729, 8)
(23730, 8)
(23731, 8)
(23732, 8)
(23733, 8)
(23734, 8)
(23735, 8)
(23736, 8)
(23737, 8)
(23738, 8)
(23739, 8)
(23740, 8)
(23741, 8)
(23742, 8)
(23743, 8)
(23744, 8)
(23745, 8)
(23746, 8)
(23747, 8)
(23748, 8)
(23749, 8)
(23750, 8)
(23751, 8)
(23752, 8)
(23753, 8)

(24426, 8)
(24427, 8)
(24428, 8)
(24429, 8)
(24430, 8)
(24431, 8)
(24432, 8)
(24433, 8)
(24434, 8)
(24435, 8)
(24436, 8)
(24437, 8)
(24438, 8)
(24439, 8)
(24440, 8)
(24441, 8)
(24442, 8)
(24443, 8)
(24444, 8)
(24445, 8)
(24446, 8)
(24447, 8)
(24448, 8)
(24449, 8)
(24450, 8)
(24451, 8)
(24452, 8)
(24453, 8)
(24454, 8)
(24455, 8)
(24456, 8)
(24457, 8)
(24458, 8)
(24459, 8)
(24460, 8)
(24461, 8)
(24462, 8)
(24463, 8)
(24464, 8)
(24465, 8)
(24466, 8)
(24467, 8)
(24468, 8)
(24469, 8)
(24470, 8)
(24471, 8)
(24472, 8)
(24473, 8)
(24474, 8)
(24475, 8)
(24476, 8)
(24477, 8)
(24478, 8)
(24479, 8)
(24480, 8)
(24481, 8)
(24482, 8)
(24483, 8)
(24484, 8)
(24485, 8)
(24486, 8)
(24487, 8)
(24488, 8)
(24489, 8)
(24490, 8)
(24491, 8)
(24492, 8)
(24493, 8)
(24494, 8)
(24495, 8)
(24496, 8)
(24497, 8)
(24498, 8)
(24499, 8)
(24500, 8)
(24501, 8)
(24502, 8)
(24503, 8)
(24504, 8)
(24505, 8)
(24506, 8)
(24507, 8)
(24508, 8)
(24509, 8)
(24510, 8)
(24511, 8)
(24512, 8)
(24513, 8)
(24514, 8)
(24515, 8)
(24516, 8)

(25188, 8)
(25189, 8)
(25190, 8)
(25191, 8)
(25192, 8)
(25193, 8)
(25194, 8)
(25195, 8)
(25196, 8)
(25197, 8)
(25198, 8)
(25199, 8)
(25200, 8)
(25201, 8)
(25202, 8)
(25203, 8)
(25204, 8)
(25205, 8)
(25206, 8)
(25207, 8)
(25208, 8)
(25209, 8)
(25210, 8)
(25211, 8)
(25212, 8)
(25213, 8)
(25214, 8)
(25215, 8)
(25216, 8)
(25217, 8)
(25218, 8)
(25219, 8)
(25220, 8)
(25221, 8)
(25222, 8)
(25223, 8)
(25224, 8)
(25225, 8)
(25226, 8)
(25227, 8)
(25228, 8)
(25229, 8)
(25230, 8)
(25231, 8)
(25232, 8)
(25233, 8)
(25234, 8)
(25235, 8)
(25236, 8)
(25237, 8)
(25238, 8)
(25239, 8)
(25240, 8)
(25241, 8)
(25242, 8)
(25243, 8)
(25244, 8)
(25245, 8)
(25246, 8)
(25247, 8)
(25248, 8)
(25249, 8)
(25250, 8)
(25251, 8)
(25252, 8)
(25253, 8)
(25254, 8)
(25255, 8)
(25256, 8)
(25257, 8)
(25258, 8)
(25259, 8)
(25260, 8)
(25261, 8)
(25262, 8)
(25263, 8)
(25264, 8)
(25265, 8)
(25266, 8)
(25267, 8)
(25268, 8)
(25269, 8)
(25270, 8)
(25271, 8)
(25272, 8)
(25273, 8)
(25274, 8)
(25275, 8)
(25276, 8)
(25277, 8)
(25278, 8)

(25934, 8)
(25935, 8)
(25936, 8)
(25937, 8)
(25938, 8)
(25939, 8)
(25940, 8)
(25941, 8)
(25942, 8)
(25943, 8)
(25944, 8)
(25945, 8)
(25946, 8)
(25947, 8)
(25948, 8)
(25949, 8)
(25950, 8)
(25951, 8)
(25952, 8)
(25953, 8)
(25954, 8)
(25955, 8)
(25956, 8)
(25957, 8)
(25958, 8)
(25959, 8)
(25960, 8)
(25961, 8)
(25962, 8)
(25963, 8)
(25964, 8)
(25965, 8)
(25966, 8)
(25967, 8)
(25968, 8)
(25969, 8)
(25970, 8)
(25971, 8)
(25972, 8)
(25973, 8)
(25974, 8)
(25975, 8)
(25976, 8)
(25977, 8)
(25978, 8)
(25979, 8)
(25980, 8)
(25981, 8)
(25982, 8)
(25983, 8)
(25984, 8)
(25985, 8)
(25986, 8)
(25987, 8)
(25988, 8)
(25989, 8)
(25990, 8)
(25991, 8)
(25992, 8)
(25993, 8)
(25994, 8)
(25995, 8)
(25996, 8)
(25997, 8)
(25998, 8)
(25999, 8)
(26000, 8)
(26001, 8)
(26002, 8)
(26003, 8)
(26004, 8)
(26005, 8)
(26006, 8)
(26007, 8)
(26008, 8)
(26009, 8)
(26010, 8)
(26011, 8)
(26012, 8)
(26013, 8)
(26014, 8)
(26015, 8)
(26016, 8)
(26017, 8)
(26018, 8)
(26019, 8)
(26020, 8)
(26021, 8)
(26022, 8)
(26023, 8)
(26024, 8)

(26682, 8)
(26683, 8)
(26684, 8)
(26685, 8)
(26686, 8)
(26687, 8)
(26688, 8)
(26689, 8)
(26690, 8)
(26691, 8)
(26692, 8)
(26693, 8)
(26694, 8)
(26695, 8)
(26696, 8)
(26697, 8)
(26698, 8)
(26699, 8)
(26700, 8)
(26701, 8)
(26702, 8)
(26703, 8)
(26704, 8)
(26705, 8)
(26706, 8)
(26707, 8)
(26708, 8)
(26709, 8)
(26710, 8)
(26711, 8)
(26712, 8)
(26713, 8)
(26714, 8)
(26715, 8)
(26716, 8)
(26717, 8)
(26718, 8)
(26719, 8)
(26720, 8)
(26721, 8)
(26722, 8)
(26723, 8)
(26724, 8)
(26725, 8)
(26726, 8)
(26727, 8)
(26728, 8)
(26729, 8)
(26730, 8)
(26731, 8)
(26732, 8)
(26733, 8)
(26734, 8)
(26735, 8)
(26736, 8)
(26737, 8)
(26738, 8)
(26739, 8)
(26740, 8)
(26741, 8)
(26742, 8)
(26743, 8)
(26744, 8)
(26745, 8)
(26746, 8)
(26747, 8)
(26748, 8)
(26749, 8)
(26750, 8)
(26751, 8)
(26752, 8)
(26753, 8)
(26754, 8)
(26755, 8)
(26756, 8)
(26757, 8)
(26758, 8)
(26759, 8)
(26760, 8)
(26761, 8)
(26762, 8)
(26763, 8)
(26764, 8)
(26765, 8)
(26766, 8)
(26767, 8)
(26768, 8)
(26769, 8)
(26770, 8)
(26771, 8)
(26772, 8)

In [58]:
print("Synthetic records needed: {}".format(n_synthetic_required_for(3))

17804

### 4-Anonymous Results

In [ ]:
synth_k4_df = synthetic_df(df, ['user_id', 'completed', 'explored'], quasi_list, 4)
print_status_stats(synth_k4_df)

In [ ]:
print("Synthetic records needed: {}".format(n_synthetic_required_for(4))

### 5-Anonymous Results

In [ ]:
synth_k5_df = synthetic_df(df, ['user_id', 'completed', 'explored'], quasi_list, 5)
print_status_stats(synth_k5_df)

In [ ]:
print("Synthetic records needed: {}".format(n_synthetic_required_for(5))

## Question 5

We've decided to do generalization on each of the quasi-identifiers first, and then suppress any additional records that need it. This isn't a particularly sophisticated technique, and it's certainly not the optimization that was performed in Angiuli and Waldo's "Statistical Tradeoffs ...".

Here are the generalization strategies we're using:
  * Drop `cc_by_ip` as it's largely redundant with `countryLabel`
  * Starting with the most specific, set any geographic field with fewer than 5000 records to "NA"
  * Change `YoB` to `birth_decade` which is the first three digits of the year
  * Blur all the `nforum` variables together by summing them into a `nforum_actions` variable
  * Change all the `nforum` varibles to booleans indicating whether they are greater than zero
  * Change `email_domain` to only the first three letters
  * Change `LoE` to `attended_college`; a boolean indicating if the student went to college
  
Changing the names of some of these attributes means we need a new set of quasi-identifiers to work from:

In [82]:
GENERALIZED_QUASI_IDS = [
    'postalCode',
    'city',
    'subdivision',
    'region',
    'countryLabel',
    'continent',
    'email_domain',
    'course_id',
    'mode',
    'gender',
    'nforum_activites',
    'nforum_posts_>0',
    'nforum_votes_>0',
    'nforum_endorsed_>0',
    'nforum_threads_>0',
    'nforum_comments_>0',
    'nforum_pinned_>0',
    'nforum_events_>0',
    'birth_decade',
    'attended_college'
]

### Useful Functions

Here we write all our functions for generalization, blurring, suppression, and presentation of results.

In [83]:
def generalize_geo(df, min_per_geo=5000):
    g = df.copy()
    GEO_HIERARCHY = [
        'postalCode',
        'city',
        'subdivision',
        'region',
        'countryLabel',
        'continent']
    g = g.drop('cc_by_ip', axis=1)
    for field in GEO_HIERARCHY:
        value_counts = g[field].value_counts()
        values_to_drop = set(value_counts[value_counts < min_per_geo].index.values)
        g.loc[g[field].isin(values_to_drop), field] = na_replacements[field]
    return g

In [84]:
def generalize_education(df):
    g = df.copy()
    # Associate is sometimes not included as college, but we are including it.
    college_levels = set(['p', 'p_se', 'p_oth', 'm', 'b', 'a'])
    g['attended_college'] = g['LoE'].isin(college_levels)
    return g.drop('LoE', axis=1)

In [85]:
def generalize_birth_year(df):
    g = df.copy()
    g['birth_decade'] = df['YoB'].apply(lambda y: str(y)[:3] + '*' if y>-1 else 'NA')
    return g.drop('YoB', axis=1)

In [ ]:
def blur_forum_actions(df):
    g = df.copy()
    g['forum_actions'] = g['nforum_posts'] + g['nforum_votes'] + g['nforum_endorsed'] + g['nforum_threads'] + g['nforum_comments'] + g['nforum_pinned'] + g['nforum_events']
    return g

In [86]:
def generalize_forum_actions(df):
    g = df.copy()
    fields = ['nforum_posts',
        'nforum_votes',
        'nforum_endorsed',
        'nforum_threads',
        'nforum_comments',
        'nforum_pinned',
        'nforum_events']
    for field in fields:
        g[field + '_>0'] = g[field] > 0
        g = g.drop(field, axis=1)
    return g

In [87]:
def generalize_email_domains(df):
    g = df.copy()
    g['email_domain'] = g['email_domain'].apply(lambda e: e[:3] + '*')
    return g

In [88]:
def generalize_df(df):
    return generalize_geo(
        generalize_birth_year(
        generalize_education(
        generalize_email_domains(blur_forum_actions(
        generalize_forum_actions(df))))))

In [111]:
def suppress_to_k_anonymize(df, quasi_ids, k):
    return df.groupby(quasi_ids).filter(lambda g: g.shape[0] >= k)

In [73]:
def status_count(df, status):
    return df.query(status + ' == True').shape[0]

def status_proportion(df, status):
    return status_count(df, status) / df.shape[0] * 100

def print_status_stats(df):
    print(f'Total students: {df.shape[0]}\n'
          f'Students explored: {status_count(df, "explored")}'
          f' - {status_proportion(df, "explored"):.2f}%\n'
          f'Students completed: {status_count(df, "completed")}'
          f' - {status_proportion(df, "completed"):.2f}%')

### Raw Dataset Stats

In [75]:
print_status_stats(df)

Total students: 1141735
Students explored: 122258 - 10.71%
Students completed: 28142 - 2.46%


### 3-Anonymous Results

In [ ]:
generalized = generalize_df(df)

In [112]:
k3_df = suppress_to_k_anonymize(generalized, GENERALIZED_QUASI_IDS, 3)

In [114]:
print_status_stats(k3_df)

Total students: 596580
Students explored: 41138 - 6.90%
Students completed: 4120 - 0.69%


### 4-Anonymous Results

In [115]:
k4_df = suppress_to_k_anonymize(generalized, GENERALIZED_QUASI_IDS, 4)

In [116]:
print_status_stats(k4_df)

Total students: 531984
Students explored: 34513 - 6.49%
Students completed: 3102 - 0.58%


### 5-Anonymous Results

In [118]:
k5_df = suppress_to_k_anonymize(generalized, GENERALIZED_QUASI_IDS, 5)

In [119]:
print_status_stats(k5_df)

Total students: 487536
Students explored: 30230 - 6.20%
Students completed: 2492 - 0.51%


### Observations

We can see that there's a steep dropoff in the number and proportion of students that completed and explored the class between the raw dataset and the 3-anonymous set. Specifically, in the raw data 2.46% of students completed, whereas in the 3-anonymous dataset, only 0.69% of students completed. As we increase the level of k-anonymity, there are fewer students overall and the proportion of students that completed and explored continues to drop off. This demonstrates that making a dataset k-anonymous may seriously impact the statistical validity of the dataset.

## Question 6

After reviewing all the field, we believe the following could be sensitive: `completed`, `grade`, `nproblem_check`, `nshow_answer`, `cert_status`. To check for the l-diversity, we're taking the minimum number of unique values per field for each quasi-identifier group.

In [120]:
def calc_l_diversity(df, field):
    return df.groupby(GENERALIZED_QUASI_IDS).apply(lambda g: g[field].nunique(dropna=False)).min()

In [122]:
sensitive_fields = ['completed', 'grade', 'nproblem_check', 'nshow_answer', 'cert_status']
k3_l_diversity = {field: calc_l_diversity(k3_df, field) for field in sensitive_fields}
k4_l_diversity = {field: calc_l_diversity(k4_df, field) for field in sensitive_fields}
k5_l_diversity = {field: calc_l_diversity(k5_df, field) for field in sensitive_fields}

### 3-Anonymous l-diversity Results

In [123]:
k3_l_diversity

{'completed': 1,
 'grade': 1,
 'nproblem_check': 1,
 'nshow_answer': 1,
 'cert_status': 1}

### 4-Anonymous l-diversity Results

In [124]:
k4_l_diversity

{'completed': 1,
 'grade': 1,
 'nproblem_check': 1,
 'nshow_answer': 1,
 'cert_status': 1}

### 5-Anonymous l-diversity Results

In [125]:
k5_l_diversity

{'completed': 1,
 'grade': 1,
 'nproblem_check': 1,
 'nshow_answer': 1,
 'cert_status': 1}

### Observations

From the above results, we can see that after all our work, none of our datasets rise above 1-diversity for any of the fields we consider sensitive. The improve this, we would need to either consider suppressing entire quasi-identifier columns or creating synthetic records to add diversity to each quasi-identifier group.